The goal is to recommend 3 similar customers based on profiles and transaction history.



In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

transactions = pd.read_csv("Transactions.csv")
customers = pd.read_csv("Customers.csv")

customer_data = transactions.groupby("CustomerID").agg(
    total_transactions=("TransactionID", "count"),
    total_quantity=("Quantity", "sum"),
    total_spent=("TotalValue", "sum")
).reset_index()

customer_profiles = pd.merge(customers, customer_data, on="CustomerID", how="left").fillna(0)

scaler = StandardScaler()
features = scaler.fit_transform(customer_profiles[["total_transactions", "total_quantity", "total_spent"]])

similarity_matrix = cosine_similarity(features)

lookalike_results = {}
for i, customer in enumerate(customer_profiles["CustomerID"][:20]):
    similar_indices = similarity_matrix[i].argsort()[::-1][1:4]  # Get top 3 similar customers
    similar_customers = [(customer_profiles["CustomerID"].iloc[j], similarity_matrix[i][j]) for j in similar_indices]
    lookalike_results[customer] = similar_customers

lookalike_df = pd.DataFrame([
    {"cust_id": key, "similar_customers": val} for key, val in lookalike_results.items()
])
lookalike_df.to_csv("Lookalike.csv", index=False)

print("Lookalike recommendations have been saved to 'Lookalike.csv'.")


Lookalike recommendations have been saved to 'Lookalike.csv'.
